In [ ]:
# Commented out IPython magic to ensure Python compatibility.
# YOLO v3
import os
import csv
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import keras
import PIL
import struct
import cv2
import math
 
import tensorflow as tf
'''from skimage.transform import resize
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D, BatchNormalization, LeakyReLU, ZeroPadding2D, UpSampling2D
from keras.models import load_model, Model
from keras.layers.merge import add, concatenate
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib.patches import Rectangle'''
from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow
 
# %matplotlib inline

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
########data augmentation##################
# initializing image and corresponding grid dimentions
### 
grid_height = 15
grid_width = 20
unit_pixel = 8
img_height = 120
img_width = 160
 
 
 
 
### 15, 20  ,  30, 40  , 60, 80, -- >>  division
### 32, 32,   16, 16  , 8, 8   ->> block size
 
 
## vol up, vol down, back, forward, pause, play
 
## (prob. of object presence , leftup x, left up y, right down x, right down y, pc1, pc2,...)
## where object is truly , present ->> train for loss on confidence, and all other things
## where no object is present ->>   train only on loss on probability of object present
 
## construction ->> where objcts are present -> (1, ....., 1, ...)
##                  where not -> (0, 0,000000)
 
## after training
## apply IOU and for probibility of objects greater than 0.5 .
## of all the objects remaining, draw them.

In [ ]:
dct = {'play' : 0, 'stop':1, 'right': 2, 'left': 3, 'volup': 4, 'voldown': 5}
dct_rev = {0: 'play', 1:'stop', 2:'right', 3:'left', 4:'volup', 5:'voldown'}

In [ ]:
path = '/content/drive/My Drive/SOAI/Ass 3/'
 
lst = os.listdir(path+'data')
print(lst)
 
 
imgs = []
outs = []
'''
. . . . .
. . . . .
. . . . .
. . . . .
 
'''
for i in range(len(lst)):
  print(lst[i])
  csv = pd.read_csv(path +'labels/'+ lst[i] + '.csv')
  csv = csv.values
  image_names = csv[:, 5]
  coordinates = csv[:, 1:5]/4
  
  out_arr = np.zeros((15,20, 11))
 
  for j in range(len(image_names)):
    imgs.append(cv2.resize(cv2.imread(path+'data/' + lst[i] + '/' + image_names[j]) , (160, 120)))
    start_x = math.floor(coordinates[j,0]/unit_pixel + 0.5)
    start_y = math.floor(coordinates[j,1]/unit_pixel + 0.5)
    end_x = math.floor((coordinates[j,0]+coordinates[j, 2])/unit_pixel + 0.5)
    end_y = math.floor((coordinates[j,1]+coordinates[j, 3])/unit_pixel + 0.5)
    if(end_x == 20):
      end_x = 19
    if(end_y == 15):
      end_y = 14
    print(start_y, end_y, start_x, end_x, coordinates[j, :])
    out_arr[start_y:end_y+1, start_x:end_x+1, 0] = np.ones((end_y + 1 - start_y, end_x + 1 - start_x))
    out_arr[start_y:end_y+1, start_x:end_x+1, 1] = np.full((end_y + 1 - start_y, end_x + 1 - start_x), coordinates[j,0])
    out_arr[start_y:end_y+1, start_x:end_x+1, 2] = np.full((end_y + 1 - start_y, end_x + 1 - start_x), coordinates[j,1])
    out_arr[start_y:end_y+1, start_x:end_x+1, 3] = np.full((end_y + 1 - start_y, end_x + 1 - start_x), coordinates[j,2])
    out_arr[start_y:end_y+1, start_x:end_x+1, 4] = np.full((end_y + 1 - start_y, end_x + 1 - start_x), coordinates[j,3])
    k = dct[lst[i]]
    out_arr[start_y:end_y+1, start_x:end_x+1, k+5] = np.full((end_y + 1 - start_y, end_x + 1 - start_x), 1)
    outs.append(out_arr)

['voldown', 'left', 'stop', 'play', 'right', 'volup']
voldown
3 9 15 19 [121.75 20.25 27.75 48.75]
0 6 3 8 [20.25 2.0 41.75 48.75]
5 12 2 6 [14.25 38.0 36.0 54.75]
4 10 8 13 [66.0 30.75 35.0 47.75]
7 14 8 12 [62.5 52.75 36.0 60.5]
4 10 16 19 [124.25 30.25 34.0 49.75]
3 9 14 18 [110.5 26.75 35.0 48.5]
0 6 4 9 [28.0 1.25 40.5 46.0]
1 7 15 19 [123.25 7.5 36.75 50.5]
0 6 9 13 [69.0 1.5 36.0 45.0]
0 6 5 9 [38.75 2.25 34.75 49.0]
4 10 11 15 [86.5 30.5 32.0 50.5]
3 7 11 14 [91.5 22.75 19.5 32.25]
5 9 13 15 [100.75 39.25 19.0 30.5]
8 12 10 13 [81.75 64.5 21.0 34.5]
0 6 7 10 [54.75 2.25 25.25 45.0]
4 8 15 18 [120.5 30.0 22.75 35.75]
4 8 11 15 [91.5 33.75 27.5 33.5]
3 8 7 10 [54.75 25.0 27.75 42.0]
3 8 10 13 [80.0 23.25 24.0 41.0]
0 5 5 8 [39.25 2.25 25.5 37.25]
7 12 15 18 [120.0 53.25 27.25 45.0]
7 13 11 15 [86.0 58.25 33.25 45.0]
2 9 13 18 [105.0 12.75 36.0 57.5]
2 8 14 18 [110.25 16.75 30.5 46.75]
3 9 13 17 [106.25 26.75 29.25 43.0]
2 6 10 12 [76.0 12.75 18.5 38.25]
4 8 13 16 [105.75 32.5 24.

In [ ]:
imgs = np.asarray(imgs)
outs = np.asarray(outs)
print(imgs.shape, outs.shape)

(590, 120, 160, 3) (590, 15, 20, 11)


In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(imgs, outs, test_size = 0.2, random_state = 0)
print(xTrain.shape, yTrain.shape)

(472, 120, 160, 3) (472, 15, 20, 11)


In [ ]:

def _conv_block(inp, convs, skip=True):
    x = inp
    count = 0
    for conv in convs:
        if count == (len(convs) - 2) and skip:
            skip_connection = x
        count += 1
        
        if conv['stride'] > 1: x = tf.keras.layers.ZeroPadding2D(((1,0),(1,0)))(x) # peculiar padding as darknet prefer left and top
        x = tf.keras.layers.Conv2D(conv['filter'], 
                   conv['kernel'], 
                   strides=conv['stride'], 
                   padding='valid' if conv['stride'] > 1 else 'same', # peculiar padding as darknet prefer left and top
                   name='conv_' + str(conv['layer_idx']), 
                   use_bias=False if conv['bnorm'] else True)(x)
        if conv['bnorm']: x = tf.keras.layers.BatchNormalization(epsilon=0.001, name='bnorm_' + str(conv['layer_idx']))(x)
        if conv['leaky']: x = tf.keras.layers.LeakyReLU(alpha=0.1, name='leaky_' + str(conv['layer_idx']))(x)

    return tf.keras.layers.add([skip_connection, x]) if skip else x


In [ ]:

# creating the YOLO model
def make_yolov3_model():
    input_image = tf.keras.layers.Input(shape=(img_height,img_width, 3))

    # Layer  0 => 4
    x = _conv_block(input_image, [{'filter': 32, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 0},
                                  {'filter': 64, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 1},
                                  {'filter': 32, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 2},
                                  {'filter': 64, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 3}])

    # Layer  5 => 8
    x = _conv_block(x, [{'filter': 128, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 5},
                        {'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 6},
                        {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 7}])

    # Layer  9 => 11
    x = _conv_block(x, [{'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 9},
                        {'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 10}])

    # Layer 12 => 15
    x = _conv_block(x, [{'filter': 256, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 12},
                        {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 13},
                        {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 14}])

    # Layer 16 => 36
    for i in range(7):
        x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 16+i*3},
                            {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 17+i*3}])
        
    skip_36 = x
        
    # Layer 37 => 40
    x = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 37},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 38},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 39}])

    # Layer 41 => 61
    for i in range(7):
        x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 41+i*3},
                            {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 42+i*3}])
        
    skip_61 = x
        
    # Layer 62 => 65
    x = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 62},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 63},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 64}])

    # Layer 66 => 74
    for i in range(3):
        x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 66+i*3},
                            {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 67+i*3}])
        
    # Layer 75 => 79
    x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 75},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 76},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 77},
                        {'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 78},
                        {'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 79}], skip=False)

    # Layer 80 => 82
    yolo_82 = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 80},
                              {'filter':  255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 81}], skip=False)

    # Layer 83 => 86
    x = _conv_block(x, [{'filter': 11, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 84}], skip=False)
    '''
    x = UpSampling2D(2)(x)
    x = concatenate([x, skip_61])

    # Layer 87 => 91
    x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 87},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 88},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 89},
                        {'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 90},
                        {'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 91}], skip=False)

    # Layer 92 => 94
    yolo_94 = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 92},
                              {'filter': 11, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 93}], skip=False)
    """
    # Layer 95 => 98
    x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True,   'layer_idx': 96}], skip=False)
    x = UpSampling2D(2)(x)
    x = concatenate([x, skip_36])

    # Layer 99 => 106
    yolo_106 = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 99},
                               {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 100},
                               {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 101},
                               {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 102},
                               {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 103},
                               {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 104},
                               {'filter': 4, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 105}], skip=False)
    """
    '''
    prob_of_object = tf.keras.layers.Lambda(lambda x : x[:,:,:,0:1])(x)
    prob_of_object = tf.keras.layers.Activation('sigmoid')(prob_of_object)
    
    coord = tf.keras.layers.Lambda(lambda x : x[:,:,:,1:5])(x)
    
    classes = tf.keras.layers.Lambda(lambda x : x[:,:,:,5:11])(x)
    classes = tf.keras.layers.Activation('softmax')(classes)
    concat = tf.keras.layers.Concatenate(axis = -1)([prob_of_object, coord, classes])
    model = tf.keras.Model(input_image, outputs = concat)

    return model


In [ ]:
yolov3 = make_yolov3_model()
yolov3.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           [(None, 120, 160, 3) 0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 120, 160, 32) 864         input_23[0][0]                   
__________________________________________________________________________________________________
bnorm_0 (BatchNormalization)    (None, 120, 160, 32) 128         conv_0[0][0]                     
__________________________________________________________________________________________________
leaky_0 (LeakyReLU)             (None, 120, 160, 32) 0           bnorm_0[0][0]                    
______________________________________________________________________________________________

In [ ]:
array = tf.constant([[1,2,-3,4, 7], [2,-3,4,-5,6]])
mask = tf.greater(array, 0)
print(mask)
non_zero_array = tf.boolean_mask(array, mask)
print(non_zero_array)

tf.Tensor(
[[ True  True False  True  True]
 [ True False  True False  True]], shape=(2, 5), dtype=bool)
tf.Tensor([1 2 4 7 2 4 6], shape=(7,), dtype=int32)


In [ ]:
yolov3 = tf.keras.models.load_model(path+'/yolov3.hdf5')

In [ ]:
def custom_loss(y_true, y_pred):
  prob_of_object_true = y_true[:,:,:,0:1]
  prob_of_object_pred = y_pred[:,:,:,0:1]
  pred_loss = tf.keras.losses.BinaryCrossentropy()(prob_of_object_true, prob_of_object_pred)
  '''
  selected_pred = tf.math.multiply(prob_of_object_true, y_pred)
  
  selected_pred_coord = selected_pred[:,:,:,1:5]
  mse_loss = keras.losses.mse(selected_pred_coord, y_true[:,:,:,1:5])
 
  selected_pred_classes = selected_pred[:,:,:,5:]
  categorical_loss = keras.losses.categorical_crossentropy(y_true[:,:,:,5:], selected_pred_classes)
  '''
  
  true = tf.where(y_true[0, :, :, 0] > 0).numpy()
  start0 = true[0][0]
  start1 = true[0][1]
 
  end0 = true[-1][0]
  end1 = true[-1][1]
 
  coord_pred = tf.slice(y_pred, (0, start0, start1,1), (1, end0-start0+1, end1-start1+1, 4))
  coord_true = tf.slice(y_true, (0, start0, start1,1), (1, end0-start0+1, end1-start1+1, 4))
  
  coord_loss =  tf.keras.losses.MeanSquaredError()(coord_true, coord_pred)
 
  classes_pred = tf.slice(y_pred, (0, start0, start1,5), (1, end0-start0+1, end1-start1+1, 6))
  classes_true = tf.slice(y_true, (0, start0, start1,5), (1, end0-start0+1, end1-start1+1, 6))
  
  classes_loss = tf.keras.losses.CategoricalCrossentropy()(classes_true, classes_pred)
 
  assert (tf.shape(coord_loss).numpy() == tf.shape(classes_loss).numpy()).all() 
  #assert (tf.shape(coord_loss).numpy() == tf.shape(pred_loss).numpy()).all()
  
  #print(pred_loss, coord_loss, classes_loss)
 
  loss = pred_loss + coord_loss/100 + 2*classes_loss
  return loss

In [ ]:
def loss_fn_d(y_true, y_pred):
  return tf.reduce_mean(0*y_true)

In [ ]:
optimizer = tf.keras.optimizers.Adam(0.000001)
a = tf.constant(1, dtype = tf.float64)
dummy_x = tf.constant(np.zeros((1, 120, 160, 3)), dtype = tf.float32)
no_iter = 1000
for i in range(no_iter):
    with tf.GradientTape() as Tape:
        dummy_y = yolov3(dummy_x)
        loss = loss_fn_d(dummy_y, dummy_y)
    grad_total = Tape.gradient(loss, yolov3.trainable_variables)
    loss = 0
 
    for exp in range(xTrain.shape[0]):
        inp  = xTrain[exp]
        inp = np.expand_dims(inp, axis = 0)
        y_true = yTrain[exp]
        y_true = np.expand_dims(y_true, axis = 0)
        with tf.GradientTape() as Tape:
          y_pred = yolov3(inp)
          loss_temp = custom_loss(y_true, y_pred)
          loss+=loss_temp
        grad = Tape.gradient(loss_temp, yolov3.trainable_variables)
        #print(grad[0])
        for l in range(len(grad)):
           grad_total[l] += grad[l]
    print(i, loss)  
    optimizer.apply_gradients(zip(grad_total, yolov3.trainable_variables))
    yolov3.save(path+'/yolov3.hdf5')

In [ ]:

imge = xTest[8:9, :, :, :]
np.copyto(imge, xTest[10:11, :, :, :])
cv2_imshow(imge[0])

pred = yolov3(imge)
pred = pred[0,:,:,:].numpy()
img = imge[0]
ref = 0.5

for i in range(pred.shape[0]):
  for j in range(pred.shape[1]):
    conf = pred[i,j,0]
    if conf <= 0.99 :
      pass
    else:
      idx = np.argmax(pred[i,j,5:]) 
      
      name = dct_rev[idx]
      
      
      if pred[i,j,idx+5] > 0.99:
        print(idx)
        img = cv2.rectangle(img, (int(pred[i,j,1]), int(pred[i,j,2])), (int(pred[i,j,1] + pred[i,j,3]), int(pred[i,j,2] + pred[i,j,4])),  (0,255,0), 1)
cv2_imshow(img)        


### try